# Capstone Project - Finding a Place to Build a Hospital

## Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction](#introduction)
* [Data](#data)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction <a name="introduction"></a>

In this project i will  find an optimal location for a **hospital**. Specifically i will concentrate it in **Gangwon Province**, Korea, which had the lowest population density as of 2018.

Of the location information for the medical center provided by the site, i will use **the location data of private hospitals, emergency rooms, hospitals, first aid centers, and residentials areas**, aiming to find areas **as close to residential areas as possible, and farthest from medical centers**.

The establishment of additional hospitals in appropriate locations will greatly benefit spreading the use and supply of medical services.

## Data <a name="data"></a>

For the goal, i need to find:
* location of existing medical center
* location of existing residentials areas

So, i will use the location data of:
* medical centers(private hospitals, emergency rooms, hospitals, first aid centers)
* residentials areas
from Gangwon Province by using **Foursquare API**.

## Analysis <a name="analysis"></a>

### import libraries

In [63]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
# module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


### defalut setting for searching in Foursquare API

In [119]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20190630'
LIMIT = 200

# latitude and longitude of central Gangwon Province
latitude = 37.742297
longitude = 128.362962

# search quary for searching in foursquare API
search_query = ['개인 병원',  # private hospital
                '응급실', # emergency room
                '병원', # hospital
                '응급처치센터', # first aid center
                '거주지'] # residential area


radius = 1260300

In [134]:
dataframe = []

for i in range(0, 5):
    
    # Define the corresponding URL
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query[i], radius, LIMIT)
    
    # Send the GET Request and examine the results
    results = requests.get(url).json()
    
    # Get relevant part of JSON and transform it into a pandas dataframe
    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe.append(json_normalize(venues))

### result dataframe

In [246]:
# for preserving the raw data, i will use dataframe named 'df'
df = dataframe.copy()

In [247]:
df[0].head(5)

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.postalCode,location.neighborhood,location.crossStreet
0,50c1c0b5e4b0dfcd5508e67c,Vivaldi Park Ski World (개인스키보드보관소),"[{'id': '4bf58dd8d48988d1e9941735', 'name': 'S...",v-1583581068,False,37.646191,127.681972,"[{'label': 'display', 'lat': 37.64619113285477...",60931,KR,홍천군,강원도,대한민국,"[홍천군, 강원도, 대한민국]",NaN,NaN,NaN,NaN
1,4bb17888f964a5208a953ce3,건국대학교 충주병원,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581068,False,36.979072,127.928496,"[{'label': 'display', 'lat': 36.97907157265968...",93252,KR,충주시,충청북도,대한민국,"[국원대로 82, 충주시, 충청북도, 380-951, 대한민국]",국원대로 82,380-951,NaN,NaN
2,5022f8c4e4b0e522d4697271,SD병원 (송도병원),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581068,False,37.539766,127.198057,"[{'label': 'display', 'lat': 37.53976604760042...",105129,KR,NaN,NaN,대한민국,[대한민국],NaN,NaN,NaN,NaN
3,4ce38ba277ef224b7db5437b,참조은병원,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581068,False,37.410308,127.260497,"[{'label': 'display', 'lat': 37.41030838040199...",104048,KR,광주시,경기도,대한민국,"[경안로 32-4, 광주시, 경기도, 대한민국]",경안로 32-4,NaN,NaN,NaN
4,4b6f69b0f964a5208cef2ce3,Seoul Asan Medical Center (서울아산병원),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581068,False,37.526738,127.109557,"[{'label': 'display', 'lat': 37.52673830376443...",113069,KR,서울특별시,서울특별시,대한민국,"[송파구 올림픽로43길 88, 풍납2동, 서울특별시, 서울특별시, 05505, 대한민국]",송파구 올림픽로43길 88,05505,NaN,NaN


In [249]:
# the first row of df[0] is not a medical center for human so i deleted it
df[0] = df[0].drop(0).copy()
df[0] = df[0].reset_index(drop=True)
df[0].head(5)

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.postalCode,location.neighborhood,location.crossStreet
0,5022f8c4e4b0e522d4697271,SD병원 (송도병원),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581068,False,37.539766,127.198057,"[{'label': 'display', 'lat': 37.53976604760042...",105129,KR,NaN,NaN,대한민국,[대한민국],NaN,NaN,NaN,NaN
1,4ce38ba277ef224b7db5437b,참조은병원,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581068,False,37.410308,127.260497,"[{'label': 'display', 'lat': 37.41030838040199...",104048,KR,광주시,경기도,대한민국,"[경안로 32-4, 광주시, 경기도, 대한민국]",경안로 32-4,NaN,NaN,NaN
2,4b6f69b0f964a5208cef2ce3,Seoul Asan Medical Center (서울아산병원),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581068,False,37.526738,127.109557,"[{'label': 'display', 'lat': 37.52673830376443...",113069,KR,서울특별시,서울특별시,대한민국,"[송파구 올림픽로43길 88, 풍납2동, 서울특별시, 서울특별시, 05505, 대한민국]",송파구 올림픽로43길 88,05505,NaN,NaN
3,4b4f0372f964a520a3f826e3,Samsung Medical Center (삼성서울병원),"[{'id': '4bf58dd8d48988d104941735', 'name': 'M...",v-1583581068,False,37.488430,127.084748,"[{'label': 'display', 'lat': 37.48842961367128...",116198,KR,서울특별시,서울특별시,대한민국,"[강남구 일원로 81, 일원본동, 서울특별시, 서울특별시, 06351, 대한민국]",강남구 일원로 81,06351,NaN,NaN
4,4c07919a0ed3c9289fab797d,마리아플러스,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581068,False,37.493417,127.124773,"[{'label': 'display', 'lat': 37.49341719266834...",112637,KR,서울특별시,서울특별시,대한민국,"[송파구 송이로 152, 가락2동, 서울특별시, 서울특별시, 138-806, 대한민국]",송파구 송이로 152,138-806,NaN,NaN


In [250]:
df[1].head(5)

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.city,location.state,location.neighborhood,location.postalCode,location.crossStreet
0,51769c00e4b03d0882b360b8,강릉의료원 응급실,"[{'id': '4bf58dd8d48988d194941735', 'name': 'E...",v-1583581069,False,37.749096,128.888609,"[{'label': 'display', 'lat': 37.74909637068704...",46275,KR,대한민국,[대한민국],NaN,NaN,NaN,NaN,NaN,NaN
1,4e187fd3d1648b834832c8a6,연세의과대학 원주기독병원 응급실,"[{'id': '4bf58dd8d48988d194941735', 'name': 'E...",v-1583581069,False,37.347961,127.946520,"[{'label': 'display', 'lat': 37.347961, 'lng':...",57253,KR,대한민국,[대한민국],NaN,NaN,NaN,NaN,NaN,NaN
2,4b851224f964a520d44a31e3,삼성서울병원 응급실,"[{'id': '4bf58dd8d48988d194941735', 'name': 'E...",v-1583581069,False,37.487788,127.085570,"[{'label': 'display', 'lat': 37.48778820823261...",116146,KR,대한민국,"[강남구 일원로 81, 일원본동, 강남구, 서울특별시, 대한민국]",강남구 일원로 81,서울특별시,서울특별시,NaN,NaN,NaN
3,4e741f7e62e1263515d160c2,응급실 @ 키자니아,"[{'id': '4bf58dd8d48988d182941735', 'name': 'T...",v-1583581069,False,37.510964,127.096209,"[{'label': 'display', 'lat': 37.51096372684163...",114612,KR,대한민국,"[잠실3동, 대한민국]",NaN,NaN,NaN,NaN,NaN,NaN
4,4e3e13d7aeb73139a182e5a9,순천향대학교 병원 응급실,"[{'id': '4bf58dd8d48988d194941735', 'name': 'E...",v-1583581069,False,37.533731,127.004347,"[{'label': 'display', 'lat': 37.533731, 'lng':...",121992,KR,대한민국,"[용산구 대사관로 59, 한남동, 대한민국]",용산구 대사관로 59,NaN,NaN,한남동,NaN,NaN


In [251]:
# the 16th, 24th, 31th, 36th row of df[1] are not medical centers for human so i deleted it
df[1] = df[1].drop([16, 24, 31, 36]).copy()
df[1] = df[1].reset_index(drop=True)
df[1].head(5)

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress,location.address,location.city,location.state,location.neighborhood,location.postalCode,location.crossStreet
0,51769c00e4b03d0882b360b8,강릉의료원 응급실,"[{'id': '4bf58dd8d48988d194941735', 'name': 'E...",v-1583581069,False,37.749096,128.888609,"[{'label': 'display', 'lat': 37.74909637068704...",46275,KR,대한민국,[대한민국],NaN,NaN,NaN,NaN,NaN,NaN
1,4e187fd3d1648b834832c8a6,연세의과대학 원주기독병원 응급실,"[{'id': '4bf58dd8d48988d194941735', 'name': 'E...",v-1583581069,False,37.347961,127.946520,"[{'label': 'display', 'lat': 37.347961, 'lng':...",57253,KR,대한민국,[대한민국],NaN,NaN,NaN,NaN,NaN,NaN
2,4b851224f964a520d44a31e3,삼성서울병원 응급실,"[{'id': '4bf58dd8d48988d194941735', 'name': 'E...",v-1583581069,False,37.487788,127.085570,"[{'label': 'display', 'lat': 37.48778820823261...",116146,KR,대한민국,"[강남구 일원로 81, 일원본동, 강남구, 서울특별시, 대한민국]",강남구 일원로 81,서울특별시,서울특별시,NaN,NaN,NaN
3,4e741f7e62e1263515d160c2,응급실 @ 키자니아,"[{'id': '4bf58dd8d48988d182941735', 'name': 'T...",v-1583581069,False,37.510964,127.096209,"[{'label': 'display', 'lat': 37.51096372684163...",114612,KR,대한민국,"[잠실3동, 대한민국]",NaN,NaN,NaN,NaN,NaN,NaN
4,4e3e13d7aeb73139a182e5a9,순천향대학교 병원 응급실,"[{'id': '4bf58dd8d48988d194941735', 'name': 'E...",v-1583581069,False,37.533731,127.004347,"[{'label': 'display', 'lat': 37.533731, 'lng':...",121992,KR,대한민국,"[용산구 대사관로 59, 한남동, 대한민국]",용산구 대사관로 59,NaN,NaN,한남동,NaN,NaN


In [252]:
df[2].head(5)

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,location.crossStreet
0,4ba1cbfaf964a520b5cc37e3,KS병원,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581071,False,강남구 선릉로 571,37.509429,127.044216,"[{'label': 'display', 'lat': 37.50942918903274...",119122,06143,KR,서울특별시,서울특별시,대한민국,"[강남구 선릉로 571, 삼성2동, 서울특별시, 서울특별시, 06143, 대한민국]",NaN,NaN
1,5022f8c4e4b0e522d4697271,SD병원 (송도병원),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581071,False,NaN,37.539766,127.198057,"[{'label': 'display', 'lat': 37.53976604760042...",105129,NaN,KR,NaN,NaN,대한민국,[대한민국],NaN,NaN
2,4bb17888f964a5208a953ce3,건국대학교 충주병원,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581071,False,국원대로 82,36.979072,127.928496,"[{'label': 'display', 'lat': 36.97907157265968...",93252,380-951,KR,충주시,충청북도,대한민국,"[국원대로 82, 충주시, 충청북도, 380-951, 대한민국]",NaN,NaN
3,4b6f69b0f964a5208cef2ce3,Seoul Asan Medical Center (서울아산병원),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581071,False,송파구 올림픽로43길 88,37.526738,127.109557,"[{'label': 'display', 'lat': 37.52673830376443...",113069,05505,KR,서울특별시,서울특별시,대한민국,"[송파구 올림픽로43길 88, 풍납2동, 서울특별시, 서울특별시, 05505, 대한민국]",NaN,NaN
4,4c6cca7de13db60c4a2ad7b1,9988병원,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581071,False,왕십리로 269,37.558563,127.037527,"[{'label': 'display', 'lat': 37.558563, 'lng':...",118595,NaN,KR,서울특별시,서울특별시,대한민국,"[왕십리로 269, 행당1동, 성동구, 서울특별시, 대한민국]",행당1동,NaN


In [253]:
# the 23th row of df[2] is not a medical center for human so i deleted it
df[2] = df[2].drop(23).copy()
df[2] = df[2].reset_index(drop=True)
df[2].head(5)

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,location.crossStreet
0,4ba1cbfaf964a520b5cc37e3,KS병원,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581071,False,강남구 선릉로 571,37.509429,127.044216,"[{'label': 'display', 'lat': 37.50942918903274...",119122,06143,KR,서울특별시,서울특별시,대한민국,"[강남구 선릉로 571, 삼성2동, 서울특별시, 서울특별시, 06143, 대한민국]",NaN,NaN
1,5022f8c4e4b0e522d4697271,SD병원 (송도병원),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581071,False,NaN,37.539766,127.198057,"[{'label': 'display', 'lat': 37.53976604760042...",105129,NaN,KR,NaN,NaN,대한민국,[대한민국],NaN,NaN
2,4bb17888f964a5208a953ce3,건국대학교 충주병원,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581071,False,국원대로 82,36.979072,127.928496,"[{'label': 'display', 'lat': 36.97907157265968...",93252,380-951,KR,충주시,충청북도,대한민국,"[국원대로 82, 충주시, 충청북도, 380-951, 대한민국]",NaN,NaN
3,4b6f69b0f964a5208cef2ce3,Seoul Asan Medical Center (서울아산병원),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581071,False,송파구 올림픽로43길 88,37.526738,127.109557,"[{'label': 'display', 'lat': 37.52673830376443...",113069,05505,KR,서울특별시,서울특별시,대한민국,"[송파구 올림픽로43길 88, 풍납2동, 서울특별시, 서울특별시, 05505, 대한민국]",NaN,NaN
4,4c6cca7de13db60c4a2ad7b1,9988병원,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1583581071,False,왕십리로 269,37.558563,127.037527,"[{'label': 'display', 'lat': 37.558563, 'lng':...",118595,NaN,KR,서울특별시,서울특별시,대한민국,"[왕십리로 269, 행당1동, 성동구, 서울특별시, 대한민국]",행당1동,NaN


In [254]:
df[3].head(5)

""


In [255]:
df[4].head(5)

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.formattedAddress
0,4e6e56b218380b4aac609c5e,형진의 거주지,"[{'id': '4d4b7105d754a06375d81259', 'name': 'P...",v-1583581077,False,37.477375,127.056503,"[{'label': 'display', 'lat': 37.47737503051758...",118923,KR,대한민국,"[개포4동, 대한민국]"


df[3] for first aid centers and df[4] for residential areas have poor information. So i googled them.

In korea, there was no proper first aid center.
For residential areas, i searched it by keyword, '주택단지(a housing complex)', and '아파트(apartment)'.
This is the result about each.

![housing complex](./housing_complex.png)
![apartment](./apartment.png)

### Define information of interest and filter dataframe

In [175]:
# keep only columns that include venue name, and anything that is associated with location

filtered_columns = ['name', 'categories'] + [col for col in df[0].columns if col.startswith('location.')] + ['id']

df_filtered = []

for i in range(0,3):
    df_filtered.append(df[i].loc[:, filtered_columns])

In [177]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [179]:
# filter the category for each row
for i in range(0,3):
    df_filtered[i]['categories'] = df_filtered[i].apply(get_category_type, axis=1)

# clean column names by keeping only last term
for i in range(0,3):
    df_filtered[i].columns = [column.split('.')[-1] for column in df_filtered[i].columns]

In [256]:
df_filtered[0].head(5)

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,postalCode,neighborhood,crossStreet,id
0,건국대학교 충주병원,Hospital,36.979072,127.928496,"[{'label': 'display', 'lat': 36.97907157265968...",93252,KR,충주시,충청북도,대한민국,"[국원대로 82, 충주시, 충청북도, 380-951, 대한민국]",국원대로 82,380-951,NaN,NaN,4bb17888f964a5208a953ce3
1,SD병원 (송도병원),Hospital,37.539766,127.198057,"[{'label': 'display', 'lat': 37.53976604760042...",105129,KR,NaN,NaN,대한민국,[대한민국],NaN,NaN,NaN,NaN,5022f8c4e4b0e522d4697271
2,참조은병원,Hospital,37.410308,127.260497,"[{'label': 'display', 'lat': 37.41030838040199...",104048,KR,광주시,경기도,대한민국,"[경안로 32-4, 광주시, 경기도, 대한민국]",경안로 32-4,NaN,NaN,NaN,4ce38ba277ef224b7db5437b
3,Seoul Asan Medical Center (서울아산병원),Hospital,37.526738,127.109557,"[{'label': 'display', 'lat': 37.52673830376443...",113069,KR,서울특별시,서울특별시,대한민국,"[송파구 올림픽로43길 88, 풍납2동, 서울특별시, 서울특별시, 05505, 대한민국]",송파구 올림픽로43길 88,05505,NaN,NaN,4b6f69b0f964a5208cef2ce3
4,Samsung Medical Center (삼성서울병원),Medical Center,37.488430,127.084748,"[{'label': 'display', 'lat': 37.48842961367128...",116198,KR,서울특별시,서울특별시,대한민국,"[강남구 일원로 81, 일원본동, 서울특별시, 서울특별시, 06351, 대한민국]",강남구 일원로 81,06351,NaN,NaN,4b4f0372f964a520a3f826e3


In [257]:
df_filtered[1].head(5)

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,postalCode,neighborhood,crossStreet,id
0,강릉의료원 응급실,Emergency Room,37.749096,128.888609,"[{'label': 'display', 'lat': 37.74909637068704...",46275,KR,NaN,NaN,대한민국,[대한민국],NaN,NaN,NaN,NaN,51769c00e4b03d0882b360b8
1,연세의과대학 원주기독병원 응급실,Emergency Room,37.347961,127.946520,"[{'label': 'display', 'lat': 37.347961, 'lng':...",57253,KR,NaN,NaN,대한민국,[대한민국],NaN,NaN,NaN,NaN,4e187fd3d1648b834832c8a6
2,삼성서울병원 응급실,Emergency Room,37.487788,127.085570,"[{'label': 'display', 'lat': 37.48778820823261...",116146,KR,서울특별시,서울특별시,대한민국,"[강남구 일원로 81, 일원본동, 강남구, 서울특별시, 대한민국]",강남구 일원로 81,NaN,NaN,NaN,4b851224f964a520d44a31e3
3,응급실 @ 키자니아,Theme Park,37.510964,127.096209,"[{'label': 'display', 'lat': 37.51096372684163...",114612,KR,NaN,NaN,대한민국,"[잠실3동, 대한민국]",NaN,NaN,NaN,NaN,4e741f7e62e1263515d160c2
4,순천향대학교 병원 응급실,Emergency Room,37.533731,127.004347,"[{'label': 'display', 'lat': 37.533731, 'lng':...",121992,KR,NaN,NaN,대한민국,"[용산구 대사관로 59, 한남동, 대한민국]",용산구 대사관로 59,NaN,한남동,NaN,4e3e13d7aeb73139a182e5a9


In [258]:
df_filtered[2].head(5)

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,postalCode,neighborhood,crossStreet,id
0,KS병원,Hospital,37.509429,127.044216,"[{'label': 'display', 'lat': 37.50942918903274...",119122,KR,서울특별시,서울특별시,대한민국,"[강남구 선릉로 571, 삼성2동, 서울특별시, 서울특별시, 06143, 대한민국]",강남구 선릉로 571,06143,NaN,NaN,4ba1cbfaf964a520b5cc37e3
1,SD병원 (송도병원),Hospital,37.539766,127.198057,"[{'label': 'display', 'lat': 37.53976604760042...",105129,KR,NaN,NaN,대한민국,[대한민국],NaN,NaN,NaN,NaN,5022f8c4e4b0e522d4697271
2,건국대학교 충주병원,Hospital,36.979072,127.928496,"[{'label': 'display', 'lat': 36.97907157265968...",93252,KR,충주시,충청북도,대한민국,"[국원대로 82, 충주시, 충청북도, 380-951, 대한민국]",국원대로 82,380-951,NaN,NaN,4bb17888f964a5208a953ce3
3,Seoul Asan Medical Center (서울아산병원),Hospital,37.526738,127.109557,"[{'label': 'display', 'lat': 37.52673830376443...",113069,KR,서울특별시,서울특별시,대한민국,"[송파구 올림픽로43길 88, 풍납2동, 서울특별시, 서울특별시, 05505, 대한민국]",송파구 올림픽로43길 88,05505,NaN,NaN,4b6f69b0f964a5208cef2ce3
4,9988병원,Hospital,37.558563,127.037527,"[{'label': 'display', 'lat': 37.558563, 'lng':...",118595,KR,서울특별시,서울특별시,대한민국,"[왕십리로 269, 행당1동, 성동구, 서울특별시, 대한민국]",왕십리로 269,NaN,행당1동,NaN,4c6cca7de13db60c4a2ad7b1


### visualization

In [239]:
#generate map
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

In [240]:
# add the private hospitals as green circle markers
for lat, lng, label in zip(df_filtered[0].lat, df_filtered[0].lng, df_filtered[0].categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)

In [241]:
# add the emergency rooms hospitals as gray circle markers
for lat, lng, label in zip(df_filtered[1].lat, df_filtered[1].lng, df_filtered[1].categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='gray',
        popup=label,
        fill = True,
        fill_color='gray',
        fill_opacity=0.6
    ).add_to(venues_map)

In [242]:
# add the hospitals as blue circle markers
for lat, lng, label in zip(df_filtered[2].lat, df_filtered[2].lng, df_filtered[2].categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

In [243]:
venues_map

## Results and Discussion <a name="results"></a>

As seeing from the mapping results, medical centers and residential areas are mainly distributed in areas outside Gangwon Province.

They are largely identical.
There is no hospital near the apartment area located across Gangwon Province.

## Conclusion <a name="conclusion"></a>

Therefore, i concludes that to expand the use and supply of medical centers, optimum location is around 평창(Pyeongchang).